In [26]:
import json
import json
import pickle
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# %config InlineBackend.figure_format = 'retina'

# import os
# import sys
# import operator

import numpy as np
import pandas as pd
from math import log, log10
# import scipy.stats as scs
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib.dates as mdates
# %matplotlib inline
# import json

# model imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split


# # model evaluation imports
# from sklearn import model_selection, preprocessing, ensemble
# from sklearn.metrics import log_loss
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import KFold


import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

from gensim import corpora, models, similarities

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
from sklearn.manifold import MDS

from gensim.models.coherencemodel import CoherenceModel
from sklearn.decomposition import NMF


from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim

 
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [27]:
def load_df(filename):
    with open(filename, 'rb') as f:
            print("Loading picked")
            return pickle.load(f)

def pickle_df(df, filename="./df.pickle"):
    if filename:
        print("Writing to", filename)
        with open(filename, "wb") as output_file:
                pickle.dump(df, output_file, protocol=4)

In [5]:
df_reviews = load_df("df_reviews_only.pickle")

Loading picked


In [6]:
stop = stopwords.words('english')

In [7]:
df_reviews.columns


Index(['rev_description', 'rev_publishedAt', 'rev_quickTake', 'rev_rating',
       'rev_title', 'rev_age', 'rev_badge', 'rev_eyeColor', 'rev_location',
       'rev_name', 'rev_skinTone', 'rev_skinType'],
      dtype='object')

In [28]:
exclude = set(string.punctuation) 
stop = list(exclude) + stopwords.words('english')
snowball = SnowballStemmer('english')
lemma = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

def clean(doc):
    tokens = " ".join(nltk.word_tokenize(doc)).lower()
    stop_free = " ".join([i for i in tokens.split() if i not in stop])
    normalized = " ".join(snowball.stem(word) for word in stop_free.split(" "))
    return normalized

In [29]:
df_5star = df_reviews[df_reviews.rev_rating==5].rev_description

In [30]:
df_1star = df_reviews[df_reviews.rev_rating==1].rev_description

In [36]:
df_5star_clean= df_5star.sample(df_1star.shape[0]).map(clean)
df_1star_clean= df_1star.map(clean)

In [46]:
## 5-star

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=2, ngram_range = (3,4))#max_features=no_features, 
tfidf = tfidf_vectorizer.fit_transform(df_5star_clean)
frequencies = sum(tfidf).toarray()[0]

unranked_5 = pd.DataFrame(frequencies, index=tfidf_vectorizer.get_feature_names(), columns=['frequency'])
ranked_5 = unranked_5.sort_values('frequency', ascending = False)

In [47]:
## 1-star

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=2, ngram_range = (3,4))#max_features=no_features, 
tfidf = tfidf_vectorizer.fit_transform(df_1star_clean)
frequencies = sum(tfidf).toarray()[0]

unranked_1 = pd.DataFrame(frequencies, index=tfidf_vectorizer.get_feature_names(), columns=['frequency'])
ranked_1 = unranked_1.sort_values('frequency', ascending = False)

In [54]:
def for_d3(text, freq):
    return ["".join([word.capitalize() for word in text.split(" ")]), int(freq)]

In [58]:
ranked_5['idx'] = ranked_5.index

In [60]:
r5 = ranked_5.apply(lambda x: for_d3(x['idx'], x['frequency']), axis = 1)

In [63]:
r5.head()

,frequency,idx
goe long way,GoeLongWay,109
littl goe long,LittlGoeLong,86
littl goe long way,LittlGoeLongWay,86
make skin feel,MakeSkinFeel,80
leav skin feel,LeavSkinFeel,77


In [64]:
ranked_1['idx'] = ranked_1.index
r1 = ranked_1.apply(lambda x: for_d3(x['idx'], x['frequency']), axis = 1)


In [65]:
r1.head()

,frequency,idx
acn prone skin,AcnProneSkin,61
realli want like,RealliWantLike,53
realli want love,RealliWantLove,49
want love product,WantLoveProduct,36
high hope product,HighHopeProduct,35


In [66]:
r5.to_csv("ranked_5.csv",index=None)

In [67]:
r1.to_csv("ranked_1.csv", index=None)